In [14]:
import pandas as pd
import numpy as np
from scipy.stats import mode
import math
import copy

#### A small sample data set with four features: two numerical and two categorial

In [15]:
data='MPG, cylinders, HP, weight\ngood, 4, 75, light\nbad, 6, 90, medium\nbad, 4, 110, medium\nbad, 8, 175, weighty\nbad, 6, 95, medium\nbad, 4, 94, light\nbad, 4, 95, light\nbad, 8, 139, weighty\nbad, 8, 190, weighty\nbad, 8, 145, weighty\nbad, 6, 100, medium\ngood, 4, 92, medium\nbad, 6, 100, weighty\nbad, 8, 170, weighty\ngood, 4, 89, medium\ngood, 4, 65, light\nbad, 6, 85, medium\ngood, 4, 81, light\nbad, 6, 95, medium\ngood, 4, 93, light'

In [16]:
# converts a raw data string into clean X and Y matrixes
def process(data_string, numeric, target):
    """
    args:
    numeric: a list of columns names to be coerced to numeric values
    target: column name that represents target labels
    """
    # remove whitespace
    data=",".join(data_string.split(", "))
    # build dataframe header row
    cols = data.splitlines()[0].split(',')
    # build data frame from a list of strings
    data = data.splitlines()[1:]
    df = pd.DataFrame.from_dict(
    map(
        lambda line: dict(zip(cols, line.split(','))),
        data
    ))
    # coerce numeric columns into numeric data types
    for col in numeric:
        df.loc[:, col] = pd.to_numeric(df.loc[:, col], errors='coerce')
    
    # finally, return the X data matrix (dataframe), and the Y data labels (series)
    return df.drop([target], axis=1), pd.Series(df[target])

In [17]:
x_train, y_train = process(data, numeric=['HP'], target='MPG')

#### Create a class representing a single decision tree, with methods to fit training data, visualize the resultant tree, and predict binary classes on new test data

In [19]:
class Tree:
    
    def __init__(self):
        
        # instantiate the model
        self.nodes = []
    
    # x_train should be a Pandas DataFrame
    # y_train should be a Pandas Series
    def fit(self, x_train, y_train, feature_types, max_depth=4):
        
        self.features = x_train.columns.tolist()
        self.target = y_train.name
        self.target_values = y_train.unique().tolist()
        self.feature_types = feature_types
        
        # take in training data and append columns for categorized flag
        # and level labels (n=max_depth)
        self.df = pd.concat([x_train, pd.DataFrame(y_train)], axis=1)
        self.df.loc[:, 'is_categorized'] = 0
        for i in range(max_depth):
            self.df.loc[:, 'level_' + str(i+1)] = None

        # create the top-level node
        self.nodes.append(Node("Y", x_train.index.tolist(), self))
        
        # traverse the tree, building sub-nodes on the fly
        for level in range(1, max_depth + 1):
            
            # stopping criteria 1 (max depth reached)
            if level == max_depth:
                print "Fitting stopped: max depth reached."
                break
            
            # building the tree recursively
            # get all existing nodes at level i
            for node in (node for node in self.nodes if len(node.name) == level):
                if node.is_pure == 0:
                    # create new nodes and add to the tree's node list
                    self.nodes.extend(node.split())
                else:
                    pass # pure node  

            # stopping criteria 2 (fully categorized)
            if sum(self.df.is_categorized) == len(self.df):
                print "Fitting stopped: data fully categorized."
                break
         
        self.visualize()
    
    
    def visualize(self):
        
        # establish a mapping of tree node objects and their names
        node_map = dict(zip(map(lambda node: node.name, f.nodes),
                 self.nodes))
        
        depth = max(map(lambda node: len(node.name), self.nodes))
        width = 100

        # iterate through each level (zero based)
        for level in range(depth):
            name_string = ''
            rule_class_string = ''
            gain_string = ''
            elements = 2**level
            r = range(elements)
            # iterate through each element position within level (zero based)
            for position in range(elements):
                expected_name=''
                # iterate through each letter in node name
                for letter in range(level+1):
                    
                    # this process uses the index of the position in tree to
                    # determine the name of the node
                    size = elements / (2**letter)
                    lists = [r[i:i + size] for i in range(0, len(r), size)]
                    sliced_lists = lists[::2]
                    l = []
                    map(l.extend, sliced_lists)
                    
                    if position in l:
                        expected_name += 'Y'
                    else:
                        expected_name += 'N'
                
                # if the "expected node" exists, display some details on it
                if expected_name in node_map:
                    if node_map[expected_name].is_pure:
                        # add name to name line
                        name_string += (width / (elements + 1)) * ' '
                        name_string += node_map[expected_name].name
                        
                        # add class to detail line
                        rule_class_string += ((width / (elements + 1)) * ' ')[:-4]
                        rule_class_string += ('class: ' + node_map[expected_name].label)
                        
                        # add gain to gain line
                        gain_string += (width / (elements + 1)) * ' '
                        gain_string += '----'

                    else:
                        # add name to name line
                        name_string += (width / (elements + 1)) * ' '
                        name_string += node_map[expected_name].name
                        
                        # add rule to detail line
                        rule_class_string += ((width / (elements + 1)) * ' ')[:-4]
                        rule_class_string += (node_map[expected_name].rule['feature'] + \
                                              ': ' + str(node_map[expected_name].rule['value']))
                        
                        # add gain to gain line
                        gain_string += ((width / (elements + 1)) * ' ')[:-4]
                        gain_string += ('gain: ' + str(node_map[expected_name].rule['gain']))
                        
                else: # node doesnt exist, print blanks
                    name_string += (((width / (elements + 1)) * ' ') + '   ')
                    rule_class_string += (((width / (elements + 1)) * ' ') + '          ')
                    gain_string += (((width / (elements + 1)) * ' ') + '        ')
                    
            print name_string
            print rule_class_string
            print gain_string + '\n\n'
            # print space newlines with slashes
        
    
    # x_test should be a Pandas DataFrame
    def predict(self, x_test):
        
        # establish a mapping of tree node objects and their names
        node_map = dict(zip(map(lambda node: node.name, f.nodes),
                 self.nodes))
        
        # make a prediction for each observation vector
        predictions = []
        for i, row in x_test.iterrows():
            # always start at top level node
            n = 'Y'
            while True:
                # check if node at index is pure; if so, declare a label
                if node_map[n].is_pure:
                    predictions.append(node_map[n].label)
                    break # from while
                 
                f_type = node_map[n].rule['f_type']
                feature = node_map[n].rule['feature']
                value = node_map[n].rule['value']
                
                if f_type == 'discrete':
                    if row[feature] == value:
                        n += 'Y'
                    else:
                        n += 'N'
                else: # continuous
                    if row[feature] > value:
                        n += 'Y'
                    else:
                        n += 'N'
        
        # finally, add a new predicted label column to the test data, and return 
        x_test[self.target] = pd.Series(predictions)
        return x_test
    

#### Within the tree, Node objects represent decision points. Nodes assess all available decision criteria, and selects the most powerful split based on an information gain calculation (change in entropy).

In [32]:
class Node:
    
    def __init__(self, name, indices, parent_tree):
        
        self.name = name
        self.indices = indices
        self.parent_tree = parent_tree
        self.is_pure = False
        self.label = None
        self.rule = {}
        
        self.check_purity()
        # add node labels to parent tree df
        self.parent_tree.df.loc[self.indices, 'level_' + str(len(self.name))] = self.name
    
    
    def check_purity(self):
        
        uniques = self.parent_tree.df.loc[self.indices, self.parent_tree.target].unique()
        if len(uniques) == 1:
            self.is_pure = True
            # set all data at node to "is_categorized" = 1
            self.parent_tree.df.loc[self.indices, 'is_categorized'] = 1
            self.label = uniques[0]
        
    
    def info_gain(self, yes_pos, yes_neg, no_pos, no_neg):
    
        def logs(x, y):
            x = float(x)
            y = float(y)
            if x==0 and y==0:
                return 0
            elif x==0:
                return (y * math.log((y+x)/y, 2))
            elif y==0:
                return (x * math.log((x+y)/x, 2))
            else:
                return (x * math.log((x+y)/x, 2)) + (y * math.log((y+x)/y, 2))

        return (logs(yes_pos + no_pos, yes_neg + no_neg) - logs(yes_pos, yes_neg) - \
                logs(no_pos, no_neg)) / sum([yes_pos, yes_neg, no_pos, no_neg])
    
    
    def get_split_candidates(self, df):
        
        split_candidates = {'discrete':[], 'continuous':[]}
        
        # get lists of all possible split candidates
        for feature, f_type in zip(self.parent_tree.features, self.parent_tree.feature_types):
            # for discrete features, the candidates are every unique value
            if f_type == 'discrete':
                split_candidates['discrete'].extend(
                    [(feature, value) for value in df[feature].unique().tolist()])
            # for continuous features, we sort values and take midpoints between
            # consecutive observations having different labels
            else:
                df_sorted = df.sort_values(feature).reset_index()
                for i in range(len(df_sorted) - 1):
                    df_pair = df_sorted.loc[i:i+1, :].reset_index()
                    if df_pair.loc[0, self.parent_tree.target] != \
                    df_pair.loc[1, self.parent_tree.target]:
                          split_candidates['continuous'].append(
                              (feature, df_pair[feature].mean()))
                
        return split_candidates
    
    def choose_split(self, df, split_candidates):
        
        # search split candidates for highest information gain
        highest_gain = 0
        chosen_split = {}
        
        # TODO: does it matter if we use "good" or "bad" as the classifer + and - ??
        for feature, value in split_candidates['discrete']:
            gain = self.info_gain(
                # split decision: YES, observation label YES
                sum((df[feature]==value) & (df[self.parent_tree.target]== \
                                            self.parent_tree.target_values[0])),
                # split decision: YES, observation label NO
                sum((df[feature]==value) & (df[self.parent_tree.target]!= \
                                            self.parent_tree.target_values[0])),
                # split decision: NO, observation label YES
                sum((df[feature]!=value) & (df[self.parent_tree.target]== \
                                            self.parent_tree.target_values[0])),
                # split decision: NO, observation label NO
                sum((df[feature]!=value) & (df[self.parent_tree.target]!= \
                                            self.parent_tree.target_values[0]))
            )
            ### TEST ###
            print self.name
            print feature, value, gain
            ############
            if gain > highest_gain:
                highest_gain = round(gain, 3)
                chosen_split = {'f_type':'discrete',
                                'feature': feature,
                                'value': value,
                                'gain': highest_gain}
        
        for feature, value in split_candidates['continuous']:
            gain = self.info_gain(
                # split decision: YES, observation label YES
                sum((df[feature]>value) & (df[self.parent_tree.target]== \
                                           self.parent_tree.target_values[0])),
                # split decision: YES, observation label NO
                sum((df[feature]>value) & (df[self.parent_tree.target]!= \
                                           self.parent_tree.target_values[0])),
                # split decision: NO, observation label YES
                sum((df[feature]<=value) & (df[self.parent_tree.target]== \
                                            self.parent_tree.target_values[0])),
                # split decision: NO, observation label NO
                sum((df[feature]<=value) & (df[self.parent_tree.target]!= \
                                            self.parent_tree.target_values[0]))
            )
            ### TEST ###
            print self.name
            print feature, value, gain
            ############
            if gain > highest_gain:
                highest_gain = round(gain, 3)
                chosen_split = {'f_type': 'continuous',
                                'feature': feature,
                                'value': value,
                                'gain': highest_gain}
    
            return chosen_split
    
    def split(self):
        
        # create a copy of the node's domain in training data
        df = copy.deepcopy(
            self.parent_tree.df.loc[
                self.indices,
                self.parent_tree.features + [self.parent_tree.target]])
        
        # list all possible split points
        split_candidates = self.get_split_candidates(df)
        
        # search the list for highest information gain
        chosen_split = self.choose_split(df, split_candidates)
        self.rule = chosen_split
        
        # perform the split by splitting the current node's domain on the new rule
        if chosen_split['f_type'] == 'continuous':
            
            yes_index = df[df[chosen_split['feature']] > chosen_split['value']].index.tolist()
            no_index = df[df[chosen_split['feature']] <= chosen_split['value']].index.tolist()
            
            return [Node(self.name + 'Y', yes_index, self.parent_tree),
                   Node(self.name + 'N', no_index, self.parent_tree)]

        else: # discrete
            
            yes_index = df[df[chosen_split['feature']] == chosen_split['value']].index.tolist()
            no_index = df[df[chosen_split['feature']] != chosen_split['value']].index.tolist()
            
            return [Node(self.name + 'Y', yes_index, self.parent_tree),
                   Node(self.name + 'N', no_index, self.parent_tree)]
        


In [33]:
# fitting a new tree with training data; visualizing the tree
f = Tree()
f.fit(x_train, y_train, feature_types=['continuous', 'discrete', 'discrete'], max_depth=3)

Y
cylinders 4 0.468057773906
Y
cylinders 6 0.191631204007
Y
cylinders 8 0.15307795339
Y
weight light 0.191631204007
Y
weight medium 0.00580214901435
Y
weight weighty 0.191631204007
Y
HP 83.0 0.309840304716
YY
cylinders 4 0.0
YY
weight light 4.93432455389e-17
YY
weight medium 9.86864910778e-17
YY
HP 93.5 0.918295834054
Fitting stopped: data fully categorized.
                                                  Y
                                              cylinders: 4
                                              gain: 0.468


                                 YY                                 YN
                             HP: 93.5                             class: bad
                             gain: 0.918                                 ----


                    YYY                    YYN                                              
                class: bad                class: good                                                            
                    ----         

In [636]:
# try predicting new binary categories with some test data
data_test="MPG, cylinders, HP, weight\n?,4,93,weighty\n?,8,70,light\n?,6,113,medium\n?,6,95,weighty\n?,4,115,medium"

In [637]:
x_test, y_test = process(data_test, numeric=['HP'], target='MPG')

In [638]:
f.predict(x_test)

,HP,cylinders,weight,MPG
0,93,4,weighty,good
1,70,8,light,bad
2,113,6,medium,bad
3,95,6,weighty,bad
4,115,4,medium,bad
